In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date=dbutils.widgets.get("p_file_date")

In [0]:
dbutils.widgets.text("p_source_data","")
v_source_data=dbutils.widgets.get("p_source_data")

In [0]:
from pyspark.sql.types import *
lap_time_schema=StructType([StructField("raceId",IntegerType(),False),StructField("driverId",IntegerType(),True),StructField('lap',IntegerType(),True),StructField('position',IntegerType(),True),StructField('time',StringType(),True),StructField('milliseconds',IntegerType(),True)])
lap_time_df=spark.read.schema(lap_time_schema).csv(f"/mnt/varunstorage321/raw/{v_file_date}/lap_times")
display(lap_time_df)

In [0]:
lap_time_df.count()

1124

In [0]:
from pyspark.sql.functions import current_timestamp,lit
laptime_final_df=lap_time_df.withColumnRenamed("driverId",'driver_id').withColumnRenamed('raceId','race_id').withColumn('ingestion_date',current_timestamp()).withColumn('data_source',lit(v_source_data)).withColumn('file_date',lit(v_file_date))
display(laptime_final_df)

In [0]:
#laptime_final_df.write.mode("overwrite").parquet('/mnt/varunstorage321/processed/lap_times')

In [0]:
#With sql save as  Table 
# laptime_final_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.lap_times')

In [0]:
#Incremental Load
#spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [0]:
#Incremental Load
# laptime_final_df=laptime_final_df.select('driver_id','lap','position','time','milliseconds','ingestion_date','data_source','file_date','race_id')

In [0]:
# Incremental Load
# if(spark._jsparkSession.catalog().tableExists('f1_processed.lap_times')):
#     laptime_final_df.write.mode("overwrite").insertInto("f1_processed.lap_times")
# else:
#     laptime_final_df.write.mode("overwrite").partitionBy('race_id').format("parquet").saveAsTable("f1_processed.lap_times") 

In [0]:
# for delta
merge_condition="tgt.race_id=src.race_id AND tgt.driver_id=src.driver_id AND tgt.lap=src.lap"
merge_delta_data(laptime_final_df,'f1_processed','lap_times',processed_folder_path,merge_condition,'race_id')

In [0]:
#spark.read.parquet('/mnt/varunstorage321/processed/lap_times').display()

In [0]:
%sql
Select race_id, count(*)
from f1_processed.lap_times
group by race_id
order by race_id desc

race_id,count(1)
1053,1124
1052,1026
1047,1043
1046,1531
1045,1016
1044,1076
1043,1128
1042,1288
1041,1017
1040,946


In [0]:
%sql
Select * from f1_processed.lap_times
where file_date='2021-03-28'

In [0]:
dbutils.notebook.exit("Success")